In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
import pandas as pd
import datasets
from datasets import DatasetDict, Dataset
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig, AutoTokenizer
import torch.nn as nn
import torch
#from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import os

from sklearn.model_selection import train_test_split

In [6]:
targets = ['agreeableness', 'openness', 'conscientiousness', 'extraversion','neuroticism']

max_length = 4096

In [7]:
target = targets[3]
print(target) #change this for other classifcations

extraversion


In [22]:
authors_df = pd.read_parquet('PANDORA_author_profiles.parquet')
full_text = pd.read_parquet('authors_full_text.parquet')

author_full_text_join = pd.merge(authors_df, full_text, on='author', how='inner')
author_full_text_join_cleaned = author_full_text_join.dropna(subset=[target])

In [12]:
reddit_complete = pd.read_parquet('sample_comments/reddit_complete_NO_LANG.parquet') #consider the full text for each author

In [ ]:
import pandas as pd
reddit_complete = pd.read_parquet('sample_comments/reddit_complete_NO_LANG.parquet')

authors = []
texts = []

# Iterate over each unique author in the DataFrame
for author in set(reddit_complete.author):
    # Concatenate all texts for the current author
    full_text = ' '.join(reddit_complete[reddit_complete.author == author].text)
    
    # Append author and their full concatenated text to the lists
    authors.append(author)
    texts.append(full_text)

# Create DataFrame with authors and concatenated texts
df_full_text = pd.DataFrame({
    'author': authors,
    'full_text': texts
})

df_full_text.to_parquet('authors_generated_full_text.parquet', index=False)

In [31]:
def divide_text_into_batches(text, batch_size=max_length * 5):
    words = text.split()
    
    batches = [words[i:i + batch_size] for i in range(0, len(words), batch_size)]
    
    batches = [' '.join(batch) for batch in batches]
    
    return batches

In [32]:
texts = []
y_values = []
authors = []

for i, row in author_full_text_join_cleaned.iterrows():
    batches = divide_text_into_batches(row['full_text'])
    target_values = [row[target] for x in range(len(batches))]
    texts += batches
    authors += [row['author'] for x in range(len(batches))]
    y_values += target_values
    

In [33]:
bins = [0, 20, 40, 60, 80, 101]  # Note: upper bound of the last bin is 101 to include 100
labels = [0, 1, 2, 3, 4]

# Classify the values into bins
categories = np.digitize(y_values, bins, right=False) - 1  # -1 to shift to zero-based indexing

# Map bin indices to labels
classified_values = [labels[i] for i in categories]

y = classified_values

In [34]:
X = texts
y = y

In [35]:
len(y)

6818

In [36]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y) , test_size=0.3, stratify=y, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(np.array(X_train), np.array(y_train) , test_size=0.1, stratify=y_train, random_state=42)

In [37]:
ds = DatasetDict()

ds['test'] = Dataset.from_pandas(pd.DataFrame({'text' : np.array(X_test), 'label' : np.array(y_test)}))


In [39]:

model_base = LongformerForSequenceClassification.from_pretrained(f'PwNzDust/{target}_model', token = 'token_here',
                                                           gradient_checkpointing=False,
                                                           attention_window = 512,
                                                           num_labels = len(set(y)),
                                                           max_length = max_length)



tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = max_length, model_max_length = max_length)

/home/acascione/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
# Tokenize the input texts
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = max_length)


In [41]:
test_data = ds['test'].map(tokenization, batched=False)

Map:   0%|          | 0/2046 [00:00<?, ? examples/s]

In [42]:
auths_test = [auth for auth, text in zip(authors, texts) if text in X_test]

In [43]:
test_data = test_data.remove_columns(['text'])

In [44]:
test_data.set_format('torch')

In [47]:
# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_base.to(device)


LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

In [48]:
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score

In [49]:
# Function to make predictions without batching
def predict(model, dataset):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(len(dataset)):
            inputs = {key: dataset[i][key].unsqueeze(0).to(model.device) for key in dataset[i] if key != 'label'}
            outputs = model(**inputs)
            preds = torch.argmax(outputs.logits, dim=-1)
            predictions.append(preds.item())
            #print(preds)
    return predictions

# Make predictions on the test data
predictions = predict(model_base, test_data)

Initializing global attention on CLS token...


In [50]:
df_evaluation = pd.DataFrame({'text' : X_test, 'ground_truth' : y_test , 'pred' : predictions, 'author' : auths_test})

In [51]:
df_evaluation

,text,ground_truth,pred,author
0,jackboot-thug US civilian murdering American m...,4,4,-BigSexy-
1,fr as being a sexist.6. Thierry Baudet says th...,2,0,-BigSexy-
2,stack. Until we get the ultimate truth of the ...,2,2,-BigSexy-
3,"for you, my dear. We won't let those monsters ...",2,3,-CrestiaBell
4,"down as of noon GMT.~~Not down, just extremely...",4,4,-CrestiaBell
...,...,...,...,...
2041,of evidence relevant to me.and they're still d...,0,0,pungens
2042,that is the domain of the FA (and FIFA to an e...,0,0,quakeroaks
2043,"on account of immature/bad parents, which is n...",0,0,quakeroaks
2044,"is pretty good too OH okay i understand now, h...",1,1,rrgjl


In [59]:
df_evaluation = pd.DataFrame({'text' : X_test, 'ground_truth' : y_test , 'pred' : predictions, 'author' : auths_test})

In [60]:
# Group by 'author' and calculate mode of 'ground-truth' and 'pred'
mode_results = df_evaluation.groupby('author').agg(
    ground_truth_mode=('ground_truth', lambda x: x.mode()[0]),
    pred_mode=('pred', lambda x: x.mode()[0])
)


In [61]:
preds = np.array(model_base.pred_mode)
labels = np.array(model_base.ground_truth_mode)


precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
acc = accuracy_score(labels, preds)

In [62]:
df = pd.DataFrame({
    'accuracy': [acc],
    'f1': [f1],
    'precision': [precision],
    'recall': [recall]
})

In [63]:
df

,accuracy,f1,precision,recall
0,0.735589,0.733535,0.736087,0.735589
